In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets,linear_model

In [4]:
housing=pd.read_csv("HousingData.csv")

In [5]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(housing,test_size=.2,random_state=42)
print(len(train_data),"",len(test_data),"",len(housing))

404  102  506


In [6]:
print(train_data)

         CRIM    ZN  INDUS  CHAS     NOX     RM   AGE     DIS  RAD  TAX  \
477  15.02340   0.0  18.10   0.0  0.6140  5.304  97.3  2.1007   24  666   
15    0.62739   0.0   8.14   0.0  0.5380  5.834  56.5  4.4986    4  307   
332   0.03466   NaN   6.06   0.0  0.4379  6.031  23.3  6.6407    1  304   
423   7.05042   0.0  18.10   0.0  0.6140  6.103   NaN  2.0218   24  666   
19    0.72580   0.0   8.14   0.0  0.5380  5.727  69.5  3.7965    4  307   
..        ...   ...    ...   ...     ...    ...   ...     ...  ...  ...   
106   0.17120   0.0   8.56   0.0  0.5200  5.836  91.9  2.2110    5  384   
270   0.29916  20.0   6.96   0.0  0.4640  5.856  42.1  4.4290    3  223   
348   0.01501  80.0   2.01   0.0  0.4350  6.635  29.7  8.3440    4  280   
435  11.16040   0.0  18.10   0.0  0.7400  6.629  94.6  2.1247   24  666   
102   0.22876   0.0   8.56   0.0  0.5200  6.405  85.4  2.7147    5  384   

     PTRATIO       B  LSTAT  MEDV  
477     20.2  349.48  24.91  12.0  
15      21.0  395.62   8.47

In [7]:
housing=train_data.copy()
print(np.shape(train_data))

(404, 14)


In [9]:
matrix_corr=housing.corr()
# print(matrix_corr)
matrix_corr['MEDV'].sort_values(ascending=False)

MEDV       1.000000
RM         0.708231
B          0.339918
ZN         0.318554
DIS        0.235114
CHAS       0.199610
AGE       -0.359495
CRIM      -0.383800
RAD       -0.387467
NOX       -0.415768
TAX       -0.459795
INDUS     -0.465662
PTRATIO   -0.491210
LSTAT     -0.732602
Name: MEDV, dtype: float64

Trying out Attribute combination

In [12]:
housing['TAXRM']=housing['TAX']/housing['RM']

In [14]:
matrix_corr=housing.corr()
matrix_corr['MEDV'].sort_values(ascending=False)

MEDV       1.000000
RM         0.708231
B          0.339918
ZN         0.318554
DIS        0.235114
CHAS       0.199610
AGE       -0.359495
CRIM      -0.383800
RAD       -0.387467
NOX       -0.415768
TAX       -0.459795
INDUS     -0.465662
PTRATIO   -0.491210
TAXRM     -0.540606
LSTAT     -0.732602
Name: MEDV, dtype: float64

                         Seperating labels from Data

In [18]:
housing=train_data.drop("MEDV",axis=1)
housing_labels=train_data["MEDV"].copy()

                      Coping medium value to NA of "RM"  

In [25]:
from sklearn.impute import SimpleImputer
# imputer=SimpleImputer(strategy="median")
# imputer.fit(housing)

                                         Skcitk-learn Design

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [32]:
my_pipeLine=Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ('std_scl',StandardScaler())
])

In [33]:
housing_num=my_pipeLine.fit_transform(housing)

                                Model Training

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# model=LinearRegression()
# model=DecisionTreeRegressor()
model=RandomForestRegressor()
model.fit(housing_num,housing_labels)

RandomForestRegressor()

In [35]:
some_data=housing.iloc[:5]
some_labels=housing_labels.iloc[:5]
prepared_data=my_pipeLine.fit_transform(some_data)
model.predict(prepared_data)

array([ 9.14 , 22.728, 30.268, 26.074, 20.583])

In [36]:
list(some_labels)

[12.0, 19.9, 19.4, 13.4, 18.2]

                         Evaluating Error

In [37]:
from sklearn.metrics import mean_squared_error
housing_prediction=model.predict(housing_num)

lin_mse=mean_squared_error(housing_labels,housing_prediction)
lin_rmse=np.sqrt(lin_mse)

In [38]:
lin_rmse

1.5255120240901312

                                Cross-Validation

In [39]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(model,housing_num,housing_labels,scoring="neg_mean_squared_error",cv=10)
rmse_score=np.sqrt(-score)

In [40]:
rmse_score

array([2.79634848, 3.37701979, 3.62249882, 5.61954015, 4.36513582,
       4.04074209, 4.36791817, 2.48758531, 2.52948857, 4.84586832])

In [41]:
def anal_score(score):
    print("Score", score)
    print("Mean",score.mean())
    print("Standard_devaition",score.std())
    

In [42]:
anal_score(rmse_score)

Score [2.79634848 3.37701979 3.62249882 5.61954015 4.36513582 4.04074209
 4.36791817 2.48758531 2.52948857 4.84586832]
Mean 3.805214552581785
Standard_devaition 0.9827781131122569


                           Saving the Model

In [43]:
from joblib import dump,load
dump(model,'Dragon.joblib')

['Dragon.joblib']

In [44]:
X_test=test_data.drop("MEDV",axis=1)
Y_test=test_data["MEDV"].copy()
X_text_prepared=my_pipeLine.fit_transform(X_test)
final_predict=model.predict(X_text_prepared)
final_rms=mean_squared_error(Y_test,final_predict)
final_rmse=np.sqrt(final_rms)
print(final_predict,list(Y_test))

[25.816 34.207 17.596 24.567 17.247 21.275 20.003 16.954 21.629 20.94
 20.943 21.236  8.518 21.691 19.63  27.351 18.608  8.03  45.907 14.647
 27.784 27.488 15.869 24.45  14.669 15.287 20.711 16.45  20.519 20.668
 19.885 26.01  30.521 19.927 15.759 16.662 35.104 25.259 20.982 22.2
 19.759 33.019 46.18  18.661 27.083 13.64  16.776 26.123 18.505 30.133
 20.956 35.696 17.343 26.884 46.651 23.592 15.973 35.031 25.834 20.112
 29.08  43.783 30.282 18.884 29.402 19.566 13.591 23.989 31.632 15.41
 20.397 26.656  9.428 23.433 22.957  7.231 19.692 46.23   9.818 14.966
 20.537 12.209 20.406  9.04  20.976 28.411 21.984 26.905 24.57  22.338
 21.93   7.906 19.702 19.19  25.515 19.319 39.641 12.072 12.307 17.527
 21.049 22.702] [23.6, 32.4, 13.6, 22.8, 16.1, 20.0, 17.8, 14.0, 19.6, 16.8, 21.5, 18.9, 7.0, 21.2, 18.5, 29.8, 18.8, 10.2, 50.0, 14.1, 25.2, 29.1, 12.7, 22.4, 14.2, 13.8, 20.3, 14.9, 21.7, 18.3, 23.1, 23.8, 15.0, 20.8, 19.1, 19.4, 34.7, 19.5, 24.4, 23.4, 19.7, 28.2, 50.0, 17.4, 22.6, 15.1, 13

In [45]:
final_rmse

3.367206343368866

In [46]:
prepared_data[0]

array([ 1.79167403,  0.        ,  1.21197852,  0.        ,  1.01173703,
       -1.75804373,  1.48680453, -1.00103851,  1.21786036,  1.22472125,
        0.22329795,  0.32916471,  1.31595014])